# 16 Decorators

**Decorators:** Functions that modify the behavior of other functions or classes
 
**Key Use Cases:**
 - *Logging:* Track function calls and their arguments
 - Timing: Measure execution time of functions
 - Caching/Memoization: Store results to avoid redundant calculations
 - Validation: Check inputs before function execution
 - Authentication/Authorization: Control access to functions
 - Rate limiting: Restrict how often functions can be called
 - Retry logic: Automatically retry failed operations
 - Property management: Create getter/setter methods in classes
 - Singleton pattern: Ensure only one instance of a class exists
 
**Syntax:** @decorator_name is placed above function/class definition

## Basic Decorator

In [1]:
# A decorator is a function that takes another function and extends its behavior
# without explicitly modifying it.

def my_decorator(func):
    """Wrapper function that adds behavior before and after the original function"""
    def wrapper():
        print("Something before the function")
        func()  # Call the original function
        print("Something after the function")
    return wrapper

# Apply decorator using @ syntax
@my_decorator
def say_hello():
    print("Hello!")

# Test the decorated function
say_hello()

Something before the function
Hello!
Something after the function


## Decorator with Arguments

In [2]:
# Decorators can accept arguments to control their behavior

def repeat(times):
    """Decorator factory that returns a decorator"""
    def decorator(func):
        def wrapper(*args, **kwargs):
            # Execute the function multiple times
            for _ in range(times):
                result = func(*args, **kwargs)
            return result
        return wrapper
    return decorator

@repeat(times=3)
def greet(name):
    print(f"Hello, {name}!")

# This will print the greeting 3 times
greet("Alice")

Hello, Alice!
Hello, Alice!
Hello, Alice!


## Multiple Decorators (Stacking)

In [ ]:
# You can stack multiple decorators on a single function
# They are applied from bottom to top (closest to function first)

def uppercase_decorator(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result.upper()
    return wrapper

def exclamation_decorator(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result + "!!!"
    return wrapper

# Applied order: uppercase_decorator -> exclamation_decorator
@exclamation_decorator  # Second
@uppercase_decorator    # First
def get_message(name):
    return f"hello, {name}"

print(get_message("Bob"))  # Output: HELLO, BOB!!!

## Decorator with functools.wraps

In [5]:
# functools.wraps preserves the metadata of the original function
from functools import wraps
import time

def timer_decorator(func):
    @wraps(func)  # Preserves func.__name__, func.__doc__, etc.
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"{func.__name__} took {end - start:.4f} seconds")
        return result
    return wrapper

@timer_decorator
def slow_function():
    """This function is slow"""
    time.sleep(1)
    return "Done!"

result = slow_function()
print(f"Function name: {slow_function.__name__}")
print(f"Function docstring: {slow_function.__doc__}")

slow_function took 1.0051 seconds
Function name: slow_function
Function docstring: This function is slow


## Class Decorator

In [6]:
# Decorators can also be applied to classes

def add_greeting(cls):
    """Add a greet method to a class"""
    cls.greet = lambda self: f"Hello from {self.name}"
    return cls

@add_greeting
class Person:
    def __init__(self, name):
        self.name = name

person = Person("Charlie")
print(person.greet())

Hello from Charlie


## Property Decorator (Getters/Setters)

In [8]:
# @property decorator creates managed attributes with getter and setter methods

class Circle:
    def __init__(self, radius):
        self._radius = radius  # Private attribute
    
    @property
    def radius(self):
        """Getter for radius"""
        return self._radius
    
    @radius.setter
    def radius(self, value):
        """Setter for radius with validation"""
        if value < 0:
            raise ValueError("Radius cannot be negative")
        self._radius = value
    
    @property
    def area(self):
        """Calculated property (read-only)"""
        return 3.14159 * self._radius ** 2

# Usage
circle = Circle(5)
print(f"Radius: {circle.radius}")
print(f"Area: {circle.area:.2f}")

# Update radius using setter
circle.radius = 10
print(f"New radius: {circle.radius}")
print(f"New area: {circle.area:.2f}")

Radius: 5
Area: 78.54
New radius: 10
New area: 314.16


## Practical Example: Logging Decorator

In [9]:
# Real-world decorator for logging function calls
from functools import wraps

def log_function_call(func):
    """Log function calls with arguments and return values"""
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(f"[LOG] Calling {func.__name__} with args={args}, kwargs={kwargs}")
        result = func(*args, **kwargs)
        print(f"[LOG] {func.__name__} returned {result}")
        return result
    return wrapper

@log_function_call
def add(a, b):
    return a + b

@log_function_call
def multiply(x, y):
    return x * y

result1 = add(5, 3)
result2 = multiply(4, 7)

[LOG] Calling add with args=(5, 3), kwargs={}
[LOG] add returned 8
[LOG] Calling multiply with args=(4, 7), kwargs={}
[LOG] multiply returned 28


## Practical Example: Validation Decorator

In [10]:
# Decorator for input validation
from functools import wraps

def validate_positive(func):
    """Ensure the argument is positive"""
    @wraps(func)
    def wrapper(number):
        if number < 0:
            raise ValueError("Number must be positive")
        return func(number)
    return wrapper

@validate_positive
def square_root(n):
    return n ** 0.5

# Test with valid and invalid input
try:
    print(f"Square root of 16: {square_root(16)}")
    print(f"Square root of -4: {square_root(-4)}")
except ValueError as e:
    print(f"Error: {e}")

Square root of 16: 4.0
Error: Number must be positive


## Caching Decorator (Memoization)

In [ ]:
# Use lru_cache for automatic memoization to improve performance
from functools import lru_cache

# Applies the decorator to cache all function results (no size limit). 
# It stores return values based on input arguments, avoiding recomputation for repeated calls.
@lru_cache(maxsize=None)  # Cache all results.
def fibonacci(n):
    """Calculate Fibonacci number with caching"""
    if n < 2:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

# These calls are fast due to caching
print(f"Fibonacci(10): {fibonacci(10)}")
print(f"Fibonacci(20): {fibonacci(20)}")
print(f"Cache info: {fibonacci.cache_info()}")

Fibonacci(10): 55
Fibonacci(20): 6765
Cache info: CacheInfo(hits=19, misses=21, maxsize=None, currsize=21)


## Singleton Decorator Pattern

In [12]:
# Ensure only one instance of a class exists
from functools import wraps

def singleton(cls):
    """Decorator to make a class a singleton"""
    instances = {}
    
    @wraps(cls)
    def get_instance(*args, **kwargs):
        if cls not in instances:
            instances[cls] = cls(*args, **kwargs)
        return instances[cls]
    return get_instance

@singleton
class Database:
    def __init__(self):
        print("Creating database connection...")
        self.connection = "Connected"

# Both variables point to the same instance
db1 = Database()
db2 = Database()
print(f"db1 is db2: {db1 is db2}")  # True - same instance

Creating database connection...
db1 is db2: True
